## MongoDB scratchpad

In [13]:
import requests
from pymongo import MongoClient;
import json
import pprint

In [2]:
# Local Mongo client
client = MongoClient("mongodb://localhost:27017")
db = client["nobel"]

In [3]:
json_list = []

with open("laureates.json") as laureates_file:
    laureates_json = json.load(laureates_file)
    json_list.append(laureates_json)

with open("prizes.json") as prizes_file:
    prizes_json = json.load(prizes_file)
    json_list.append(prizes_json)

In [4]:
prize_collection = db["prizes"]
prize_collection.insert_many(prizes_json)

In [5]:
laureates_collection = db["laureates"]
laureates_collection.insert_many(laureates_json)

In [6]:
db.list_collection_names()

['prizes', 'laureates']

In [7]:
db.prizes.prizes

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel'), 'prizes.prizes')

In [8]:
walter_kohn = {"firstname": "Walter", 
               "surname": "Kohn"}

db.laureates.find_one(walter_kohn)

{'_id': ObjectId('6415f5de5a2f5e03a293c194'),
 'id': '290',
 'firstname': 'Walter',
 'surname': 'Kohn',
 'born': '1923-03-09',
 'died': '2016-04-19',
 'bornCountry': 'Austria',
 'bornCountryCode': 'AT',
 'bornCity': 'Vienna',
 'diedCountry': 'USA',
 'diedCountryCode': 'US',
 'diedCity': 'Santa Barbara, CA',
 'gender': 'male',
 'prizes': [{'year': '1998',
   'category': 'chemistry',
   'share': '2',
   'motivation': '"for his development of the density-functional theory"',
   'affiliations': [{'name': 'University of California',
     'city': 'Santa Barbara, CA',
     'country': 'USA'}]}]}

In [15]:
# equivalent of doing "LIKE" query in SQL
regex = {"firstname": { "$regex" : "[w|W]alter"}}

cursor = db.laureates.find(regex)

for doc in cursor:
    pprint.pprint(doc)

{'_id': ObjectId('6415f5de5a2f5e03a293c0dc'),
 'born': '1902-02-10',
 'bornCity': 'Amoy',
 'bornCountry': 'China',
 'bornCountryCode': 'CN',
 'died': '1987-10-13',
 'diedCity': 'Seattle, WA',
 'diedCountry': 'USA',
 'diedCountryCode': 'US',
 'firstname': 'Walter Houser',
 'gender': 'male',
 'id': '67',
 'prizes': [{'affiliations': [{'city': 'Murray Hill, NJ',
                               'country': 'USA',
                               'name': 'Bell Telephone Laboratories'}],
             'category': 'physics',
             'motivation': '"for their researches on semiconductors and their '
                           'discovery of the transistor effect"',
             'share': '3',
             'year': '1956'}],
 'surname': 'Brattain'}
{'_id': ObjectId('6415f5de5a2f5e03a293c0f2'),
 'born': '1911-06-13',
 'bornCity': 'San Francisco, CA',
 'bornCountry': 'USA',
 'bornCountryCode': 'US',
 'died': '1988-09-01',
 'diedCity': 'Berkeley, CA',
 'diedCountry': 'USA',
 'diedCountryCode': 'US',


In [10]:
regex = {"firstname": { "$regex" : "[w|W]alter"}}

cursor2 = db.laureates.find(regex)

# to print things in a rich format
for doc in cursor2:
    doc["_id"] = str(doc["_id"]) # Necessary because "ObjectID()" needs to be a string to show correctly (see above)
    print(json.dumps(doc, indent=4))

{
    "_id": "6415f5de5a2f5e03a293c0dc",
    "id": "67",
    "firstname": "Walter Houser",
    "surname": "Brattain",
    "born": "1902-02-10",
    "died": "1987-10-13",
    "bornCountry": "China",
    "bornCountryCode": "CN",
    "bornCity": "Amoy",
    "diedCountry": "USA",
    "diedCountryCode": "US",
    "diedCity": "Seattle, WA",
    "gender": "male",
    "prizes": [
        {
            "year": "1956",
            "category": "physics",
            "share": "3",
            "motivation": "\"for their researches on semiconductors and their discovery of the transistor effect\"",
            "affiliations": [
                {
                    "name": "Bell Telephone Laboratories",
                    "city": "Murray Hill, NJ",
                    "country": "USA"
                }
            ]
        }
    ]
}
{
    "_id": "6415f5de5a2f5e03a293c0f2",
    "id": "89",
    "firstname": "Luis Walter",
    "surname": "Alvarez",
    "born": "1911-06-13",
    "died": "1988-09-01",
 

In [11]:
# Filter for laureates born in Austria with non-Austria prize affiliation
criteria = {"bornCountry": "Austria", 
            # below is required because affiliations is a nested 
            # array itself inside the nested array of prizes
              "prizes.affiliations.country": {"$ne": "Austria"}} 

# Count the number of such laureates
count = db.laureates.count_documents(criteria)
print(count)

10


In [14]:
# Filter for laureates with at least three prizes
# This is borrowing from JS -- "Does element 3 (index 2) exist?"
criteria = {"prizes.2": {"$exists": True}}

# Find one laureate with at least three prizes
doc = db.laureates.find_one(criteria)

# Print the document
pprint.pprint(doc)

{'_id': ObjectId('6415f5de5a2f5e03a293c152'),
 'born': '0000-00-00',
 'died': '0000-00-00',
 'firstname': 'Comité international de la Croix Rouge (International Committee '
              'of the Red Cross)',
 'gender': 'org',
 'id': '482',
 'prizes': [{'affiliations': [[]],
             'category': 'peace',
             'share': '1',
             'year': '1917'},
            {'affiliations': [[]],
             'category': 'peace',
             'share': '1',
             'year': '1944'},
            {'affiliations': [[]],
             'category': 'peace',
             'share': '2',
             'year': '1963'}]}


In [29]:
db.laureates.distinct("gender")

['female', 'male', 'org']

In [17]:
# Save a filter for laureates with unshared prizes
unshared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": "1",
        "year": {"$gte": "1945"},
    }}}

# Save a filter for laureates with shared prizes
shared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": {"$ne": "1"},
        "year": {"$gte": "1945"},
    }}}

ratio = db.laureates.count_documents(unshared) / db.laureates.count_documents(shared)
print(ratio)